In [ ]:
!pip install --quiet gTTS vaderSentiment librosa soundfile numpy

!apt-get -qq install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.9 MB/s eta 0:00:00


In [ ]:
import os
from gtts import gTTS
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import librosa
import soundfile as sf
import numpy as np
from IPython.display import Audio, display

os.makedirs("outputs", exist_ok=True)

analyzer = SentimentIntensityAnalyzer()

In [ ]:
def detect_emotion(text):
    """
    Returns: label (Positive/Neutral/Negative), compound_score (float)
    """
    scores = analyzer.polarity_scores(text)
    c = scores["compound"]
    if c >= 0.30:
        label = "Positive"
    elif c <= -0.30:
        label = "Negative"
    else:
        label = "Neutral"
    return label, c

def map_to_voice_params(label, intensity):
    """
    intensity: absolute value of compound score (0..1)
    returns: rate_factor (float), pitch_semitones (float), volume (0..1)
    rate_factor: >1 faster, <1 slower
    pitch_semitones: positive => higher pitch, negative => lower pitch
    """
    base_rate = 1.0
    base_volume = 1.0
    if label == "Positive":
        rate = base_rate + 0.20 * intensity
        pitch = +2.0 * intensity
        volume = min(1.0, base_volume + 0.1*intensity)
    elif label == "Negative":
        rate = base_rate - 0.12 * intensity
        pitch = -2.0 * intensity
        volume = max(0.8, base_volume - 0.15*intensity)
    else:
        rate = base_rate
        pitch = 0.0
        volume = base_volume

    return float(rate), float(pitch), float(volume)

In [ ]:
def synthesize_with_gtts(text, tmp_mp3_path):
    """Create an mp3 using gTTS and save to tmp_mp3_path"""
    tts = gTTS(text=text, lang='en', slow=False)
    tts.save(tmp_mp3_path)
    return tmp_mp3_path

def process_audio_change_rate_and_pitch(in_path, out_path, rate_factor=1.0, pitch_semitones=0.0, target_sr=22050):
    """
    Load audio, time-stretch (rate), pitch-shift (semitones), save wav.
    rate_factor: >1 faster, <1 slower
    pitch_semitones: positive => higher pitch
    """
    y, sr = librosa.load(in_path, sr=target_sr)

    if rate_factor <= 0:
        rate_factor = 1.0
    if abs(rate_factor - 1.0) > 0.02:
        y = librosa.effects.time_stretch(y, rate=rate_factor)

    if abs(pitch_semitones) > 0.01:
        y = librosa.effects.pitch_shift(y, sr=sr, n_steps=pitch_semitones)

    max_val = np.max(np.abs(y))
    if max_val > 0:
        y = y / max_val * 0.95

    sf.write(out_path, y, target_sr, format='WAV')
    return out_path

In [ ]:
def empathy_synthesize(text, out_basename="outputs/resp", demo=False):
    """
    Full pipeline:
      - detect emotion
      - map to voice params
      - generate TTS mp3 via gTTS
      - postprocess (rate & pitch)
      - save wav and return metadata
    """
    label, compound = detect_emotion(text)
    intensity = abs(compound)
    rate, pitch, volume = map_to_voice_params(label, intensity)

    tmp_mp3 = out_basename + "_tmp.mp3"
    raw_wav = out_basename + "_raw.wav"
    final_wav = out_basename + f"_{label.lower()}.wav"

    synthesize_with_gtts(text, tmp_mp3)

    y, sr = librosa.load(tmp_mp3, sr=22050)
    sf.write(raw_wav, y, sr)

    process_audio_change_rate_and_pitch(raw_wav, final_wav, rate_factor=rate, pitch_semitones=pitch, target_sr=22050)

    return {
        "text": text,
        "emotion_label": label,
        "compound": compound,
        "intensity": intensity,
        "rate_factor": rate,
        "pitch_semitones": pitch,
        "final_wav": final_wav
    }

demo_texts = {
    "positive": "That is fantastic news! I am really excited for you and can't wait to see the results.",
    "neutral": "Your appointment is scheduled for Thursday at 3 PM. Please bring a valid ID.",
    "negative": "I'm sorry, but we've experienced an error and cannot complete your request at this time."
}

results = {}
for k, txt in demo_texts.items():
    out = empathy_synthesize(txt, out_basename=f"outputs/demo_{k}")
    results[k] = out
    print(k, out["emotion_label"], "compound:", round(out["compound"],3), "rate:", round(out["rate_factor"],3), "pitch:", round(out["pitch_semitones"],3))

positive Positive compound: 0.764 rate: 1.153 pitch: 1.528
neutral Positive compound: 0.318 rate: 1.064 pitch: 0.636
negative Negative compound: -0.572 rate: 0.931 pitch: -1.144


In [ ]:
for k in ["positive", "neutral", "negative"]:
    path = results[k]["final_wav"]
    print(f"\n=== {k.upper()} ({results[k]['emotion_label']}) ===")
    display(Audio(path, autoplay=False))


=== POSITIVE (Positive) ===



=== NEUTRAL (Positive) ===



=== NEGATIVE (Negative) ===


In [ ]:
readme_text = """
# The Empathy Engine 🎙️
### Darwix AI Hackathon Submission

## Overview
This project is an AI-driven Text-to-Speech system that adds emotion to voice output. It detects the sentiment of a given text (Positive / Neutral / Negative) and adjusts speech rate and pitch accordingly to make it sound more natural and empathetic.

## Key Features
- Emotion detection using VADER Sentiment Analyzer.
- Dynamic vocal modulation (rate, pitch) based on detected emotion.
- Generates emotion-aligned .wav audio output.
- Offline and fully runnable in Google Colab.

## How to Run
1. Open the `Empathy_Engine.ipynb` notebook in Google Colab.
2. Run all cells (1–6).
3. Your generated outputs will appear in the `outputs/` folder.
4. The system produces three demo samples showing Positive, Neutral, and Negative speech tones.

## Libraries Used
- vaderSentiment
- gTTS
- librosa
- soundfile
- pydub

## Future Enhancements
- Add SSML for better emotion control.
- Use advanced APIs like ElevenLabs or Google Cloud TTS for expressive voices.
- Integrate a simple Flask web interface for text input and live playback.
"""

with open("README.md", "w") as f:
    f.write(readme_text)

print("✅ README.md file created successfully.")

✅ README.md file created successfully.


In [ ]:
!mkdir submission
!cp Empathy_Engine.ipynb submission/
!cp -r outputs submission/
!cp README.md submission/
!zip -r Empathy_Engine_Submission.zip submission

cp: cannot stat 'Empathy_Engine.ipynb': No such file or directory
  adding: submission/ (stored 0%)
  adding: submission/README.md (deflated 40%)
  adding: submission/outputs/ (stored 0%)
  adding: submission/outputs/demo_negative_negative.wav (deflated 18%)
  adding: submission/outputs/demo_positive_raw.wav (deflated 22%)
  adding: submission/outputs/demo_negative_raw.wav (deflated 23%)
  adding: submission/outputs/demo_neutral_tmp.mp3 (deflated 2%)
  adding: submission/outputs/demo_negative_tmp.mp3 (deflated 3%)
  adding: submission/outputs/demo_neutral_raw.wav (deflated 20%)
  adding: submission/outputs/demo_positive_positive.wav (deflated 18%)
  adding: submission/outputs/demo_neutral_positive.wav (deflated 17%)
  adding: submission/outputs/demo_positive_tmp.mp3 (deflated 2%)


In [ ]:
'Empathy_Engine_Submission.zip'

'Empathy_Engine_Submission.zip'